In [ ]:
import sys
import importlib
import datetime

from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import rich

sys.path.insert(0, '..')
from src import utils

from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from PIL import Image

importlib.reload(utils)

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')

for device in gpu_devices:
    rich.print(device)
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
full_data = Path('..', 'data', 'full_dataset.xlsx')
train_path = Path('..', 'data', 'raw_train.csv')
test_path = Path('..', 'data', 'raw_test.csv')

train_df = utils.read_raw_data(full_data, train_path, 0)
test_df = utils.read_raw_data(full_data, test_path, 1)

# shuffle the dataset
train_df = train_df.sample(frac=1, random_state=42)

X_pandas = train_df.drop(['Diet'], axis=1)
y = train_df['Diet']

train_waves = utils.convert_waves(X_pandas)

y = y.map({'GRS': 0, 'CLV': 1, 'TMR': 2}).values
train_labels = y.reshape((y.shape[0], 1))

y = tf.one_hot(y, depth=3).numpy()

T_X, test_X, T_y, test_y = train_test_split(
    train_waves, y, stratify=train_labels, test_size=0.2)

training_X, validation_X, training_y, validation_y = train_test_split(
    T_X, T_y, stratify=T_y, test_size=0.25)

rich.print(f'Training set shape: {training_X.shape}')
rich.print(f'Validation set shape: {validation_X.shape}')
rich.print(f'Test set shape: {test_X.shape}')

In [ ]:
log_dir = '../logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model = models.Sequential()

# model.add(layers.Rescaling(1./255, input_shape=(33, 33, 1)))
model.add(layers.Conv2D(16, (4, 4), activation='relu', input_shape=(33, 33, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (2, 2), activation='relu'))

model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dropout(.2))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(.2))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(.2))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(training_X,
                    training_y,
                    validation_data=(validation_X, validation_y),
                    epochs=400,
                    callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure(figsize=(10, 5))

plt.plot(history.history['accuracy'], label='train acc.')
plt.plot(history.history['val_accuracy'], label='validation acc.')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.grid()

In [ ]:
fig = plt.figure(figsize=(10, 5))

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.grid()

In [ ]:
test_loss, test_acc = model.evaluate(test_X,  test_y, verbose=2)

rich.print(f'Test accuracy: {test_acc}')